# Train a deep CNN on XPS data on Google Colab

In this notebook, we will train a deep convolutional network on iron XPS spectra using the local CPU/GPU.

## Setup

### Mount google drive, change working directory

In [ ]:
# Mount drive
from google.colab import drive
import os

drive.mount('/content/drive')

# Change working path
os.chdir('/content/drive/My Drive/app')

### Install packages and import modules

In [ ]:
# Install packages
!pip install python-docx

# Import standard modules and magic commands
import datetime
import numpy as np
import pytz
import importlib

# Magic commands
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Disable tf warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf

### Load custom modules

In [ ]:
try:
    import importlib
    importlib.reload(classifier)
    importlib.reload(clfutils)
    print('Modules were reloaded.')
except:
    import xpsdeeplearning.network.classifier as classifier
    import xpsdeeplearning.network.utils as clfutils
    print('Modules were loaded.')

## Classification

### Setting up the parameters & folder structure

In [ ]:
np.random.seed(502)
time = datetime.datetime.now().astimezone(pytz.timezone('Europe/Berlin')).strftime("%Y%m%d_%Hh%Mm")
data_name = 'Fe_single_4_classes'

label_values = ['Fe metal', 'FeO', 'Fe3O4', 'Fe2O3']

clf = classifier.ClassifierMultiple(time = time,
                                    data_name = data_name,
                                    labels = label_values)

### Load and inspect the data

In [ ]:
input_filepath = r'/content/drive/My Drive/app/datasets/20200605_iron_single_small.h5'
train_test_split = 0.2
train_val_split = 0.2
no_of_examples = 100000

X_train, X_val, X_test, y_train, y_val, y_test = \
        clf.load_data_preprocess(input_filepath = input_filepath,
                                 no_of_examples = no_of_examples,
                                 train_test_split = train_test_split,
                                 train_val_split = train_val_split)
        
# Check how the examples are distributed across the classes.
class_distribution = clf.check_class_distribution()
clf.plot_class_distribution()
clf.plot_random(no_of_spectra = 9, dataset = 'train')  

### Design the model

In [ ]:
try:
    importlib.reload(models)
    print('Models module was reloaded.')
except:
    import xpsdeeplearning.network.models as models
    print('Models module was loaded.')

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, concatenate
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling1D, MaxPooling1D
#from tensorflow.keras.layers import BatchNormalization

class CustomCNNSub(models.EmptyModel):
    def __init__(self, inputshape, num_classes, name = None):      
        input_1 = Input(shape = inputshape)
                
        conv_1_short = Conv1D(4, 5, padding = 'same',
                            activation = 'relu')(input_1)
        conv_1_medium = Conv1D(4, 10, padding = 'same',
                             activation = 'relu')(input_1)
        conv_1_long = Conv1D(4, 15, padding = 'same',
                           activation = 'relu')(input_1)
        sublayers = [conv_1_short, conv_1_medium, conv_1_long]
        merged_sublayers = concatenate(sublayers)
        
        conv_2 = Conv1D(4, 5, activation='relu')(merged_sublayers)
        average_pool_1 = AveragePooling1D()(conv_2)
        
        flatten_1 = Flatten()(average_pool_1)
        drop_1 = Dropout(0.2)(flatten_1)
        dense_1 = Dense(2000, activation = 'relu')(drop_1)
        
        dense_2 = Dense(num_classes, activation = 'softmax')(dense_1)
        
        no_of_inputs = len(sublayers)

        super(CustomCNNSub, self).__init__(inputs = input_1,
                                           outputs = dense_2,
                                           inputshape = inputshape,
                                           num_classes = num_classes,
                                           no_of_inputs = no_of_inputs, 
                                           name = 'Custom_CNN_Sub')

### Build the model

In [ ]:
clf.model = CustomCNNSub(clf.input_shape, clf.num_classes)

# Alternative: Build model from available models
#clf.model = models.CustomCNNSub(clf.input_shape, clf.num_classes)

### Compile and summarize the model

In [ ]:
from tensorflow.keras.optimizers import Adam
learning_rate = 1e-05
optimizer = Adam(learning_rate = learning_rate) 


def custom_loss(y_true, y_pred):
    """
    Custom loss for linear combination of XPS spectra.
    """
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)  # Note the `axis=-1`

#clf.model.compile(loss = custom_loss, optimizer = 'adam', )


# Alternative: Compile model with build-in loss function
clf.model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizer)


# Plot summary and save model plot.
clf.summary()
clf.save_and_print_model_image()

### Training

In [ ]:
epochs = 150
batch_size = 32

hist = clf.train(checkpoint = True,
                 early_stopping = False,
                 tb_log = True, 
                 csv_log = True,
                 epochs = epochs, 
                 batch_size = batch_size,
                 verbose = 1)

### Graphs

In [ ]:
dir_name = clf.time + '_' + clf.data_name
graph = clfutils.TrainingGraphs(clf.history, dir_name)
graph.plot_loss()

### Evaluation on test data

In [ ]:
test_loss = clf.evaluate()
print('Test loss: ' + str(np.round(test_loss, decimals=3)))

###  Prediction on test data

In [ ]:
pred_train, pred_test = clf.predict()

### Show some predictions

#### 5 random training samples

In [ ]:
clf.plot_random(no_of_spectra = 6, dataset = 'train', with_prediction = True)  

#### 5 random test samples

In [ ]:
clf.plot_random(no_of_spectra = 6, dataset = 'test', with_prediction = True)  

### Saving model and data

In [ ]:
clf.save_model()
clf.save_hyperparams()
clf.shelve_results(full = False)

### Report

In [ ]:
dir_name = clf.time + '_' + clf.data_name
rep = clfutils.Report(dir_name)  
rep.write()   

### Save output of notebook

In [ ]:
from IPython.display import Javascript, display
from nbconvert import HTMLExporter

def save_notebook():
    display(Javascript("IPython.notebook.save_notebook()"),
            include=['application/javascript'])

def output_HTML(read_file, output_file):
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is '.ipynb', output_file is '.html'
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)

import time
import os

save_notebook()
time.sleep(10)
current_file = '/content/drive/My Drive/app/xpsdeeplearning/train_colab.ipynb'
output_file = os.path.join(clf.log_dir,'train_colab_out.html')
output_HTML(current_file, output_file)